# NL→SQL Demo Notebook (Azure OpenAI + Azure SQL)

This notebook demonstrates the NL→Intent→SQL pipeline step by step, aligned with the project's flowchart.
Each step includes explanation cells for demo purposes (e.g., for a TERADATA partner presentation).

Prereqs: ensure you have a valid .env configured for Azure OpenAI and Azure SQL (see README).

In [1]:
# 1) Imports and environment bootstrap
import os, re, sys
from typing import List, Dict, Any
from dotenv import load_dotenv
load_dotenv()
print('Environment loaded. Project root:', os.getcwd())

Environment loaded. Project root: /Users/arturoquiroga/GITHUB/AQ-NEW-NL2SQL/docs


## Demo settings (global)

- Control printing and execution across the notebook from one place.
- You can still override the query text in runner cells if needed.

In [2]:
# Demo settings (global)
# These control printing and execution across the notebook.
SHOW_INTENT = True
SHOW_REASONING = True
NO_EXEC = False
REFRESH_SCHEMA = False

# Default query used by runner cells unless overridden
default_test_query = 'Weighted average interest rate by region'
print('Demo settings -> SHOW_INTENT:', SHOW_INTENT, '| SHOW_REASONING:', SHOW_REASONING, '| NO_EXEC:', NO_EXEC, '| REFRESH_SCHEMA:', REFRESH_SCHEMA)

Demo settings -> SHOW_INTENT: True | SHOW_REASONING: True | NO_EXEC: False | REFRESH_SCHEMA: False


### Display Helpers

In [3]:
# Display helpers (ANSI colors for sectioned output)
RESET = "\033[0m"
YELLOW = "\033[33m"           # intent
LIGHT_BLUE = "\033[96m"       # reasoning (bright cyan)
WHITE = "\033[97m"            # raw sql
GRAY = "\033[90m"             # legacy dark gray (not used for sanitized anymore)
LIGHT_GRAY = "\033[38;5;250m" # sanitized sql (lighter and more readable)
GOLD = "\033[38;5;220m"      # results (approx gold)

def colorize(text: str, color: str) -> str:
    return f"{color}{text}{RESET}"

def print_colored_block(label: str, content: str, color: str, sep: str = "\n") -> None:
    block = f"{label}{sep}{content}" if content is not None else label
    print(colorize(block, color))
    print()  # extra blank line for spacing between output blocks

## Azure OpenAI setup and model routing
We detect reasoning-style deployments (o-series/GPT-5) and use direct REST Chat Completions without unsupported params.
For other models, we use the LangChain AzureChatOpenAI wrapper.

In [4]:
# 2) Azure OpenAI configuration & helpers
import json, requests
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION', '2025-04-01-preview')

def _is_reasoning_like_model(deployment_name: str | None) -> bool:
    name = (deployment_name or '').lower()
    return name.startswith('o') or name.startswith('gpt-5')

USING_REASONING = _is_reasoning_like_model(DEPLOYMENT_NAME)
print('Reasoning-style deployment:', USING_REASONING, '| Deployment:', DEPLOYMENT_NAME)

def _azure_chat_completions(messages: List[dict], max_completion_tokens: int | None = None) -> str:
    url = f"{ENDPOINT.rstrip('/')}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"
    payload: Dict[str, Any] = {'messages': messages}
    if max_completion_tokens is not None:
        payload['max_completion_tokens'] = max_completion_tokens
    headers = {'api-key': API_KEY or '', 'Content-Type': 'application/json'}
    resp = requests.post(url, headers=headers, json=payload, timeout=60)
    resp.raise_for_status()
    data = resp.json()
    return data['choices'][0]['message']['content']

def _make_llm():
    if USING_REASONING:
        return None
    return AzureChatOpenAI(
        openai_api_key=API_KEY,
        azure_endpoint=ENDPOINT,
        deployment_name=DEPLOYMENT_NAME,
        api_version=API_VERSION,
        max_tokens=8192,
    )

llm = _make_llm()
print('LangChain LLM initialized:', llm is not None)

Reasoning-style deployment: True | Deployment: gpt-5
LangChain LLM initialized: False


## Prompts & function: Parse NL into intent/entities
We keep prompt strings inline to mirror the main script.

In [5]:
# 3) Prompts and parsing function
INTENT_PROMPT_TEXT = (
    "You are an expert in translating natural language to database queries.\n"
    "Extract the intent and entities from the following user input:\n"
    "{input}"
)
SQL_PROMPT_TEXT = (
    "You are an expert in SQL and Azure SQL Database. Given the following database schema and the intent/entities,\n"
    "generate a valid T-SQL query for querying the database.\n\n"
    "IMPORTANT:\n"
    "- Do NOT use USE statements (USE [database] is not supported in Azure SQL Database)\n"
    "- Generate only the SELECT query without USE or GO statements\n"
    "- Return only executable T-SQL code without markdown formatting\n"
    "- The database connection is already established to the correct database\n\n"
    "Schema:\n"
    "{schema}\n"
    "Intent and Entities:\n"
    "{intent_entities}\n\n"
    "Generate a T-SQL query that can be executed directly:\n"
)
REASONING_PROMPT_TEXT = (
    "Before writing SQL, produce a short, high-level reasoning summary for how you will construct the query,\n"
    "based on the schema and the intent/entities.\n\n"
    "Rules:\n"
    "- Do NOT include any SQL code.\n"
    "- Keep it concise (<= 150 words) and actionable.\n"
    "- Use simple bullets covering: Entities mapping, Tables/Joins, Aggregations (if any), Filters, Order/Limit, Assumptions.\n\n"
    "Schema:\n"
    "{schema}\n"
    "Intent and Entities:\n"
    "{intent_entities}\n\n"
    "Provide the reasoning summary now:\n"
)

intent_prompt = ChatPromptTemplate.from_template(INTENT_PROMPT_TEXT)
sql_prompt = ChatPromptTemplate.from_template(SQL_PROMPT_TEXT)
reasoning_prompt = ChatPromptTemplate.from_template(REASONING_PROMPT_TEXT)

def parse_nl_query(user_input: str) -> str:
    if USING_REASONING:
        prompt_text = INTENT_PROMPT_TEXT.format(input=user_input)
        messages = [{ 'role': 'user', 'content': prompt_text.strip() }]
        return _azure_chat_completions(messages, max_completion_tokens=8192)
    chain = intent_prompt | llm
    res = chain.invoke({'input': user_input})
    return res.content

print('parse_nl_query ready.')

parse_nl_query ready.


## Schema context (with optional refresh)
We use the local cache-backed context from `schema_reader.get_sql_database_schema_context`.

In [6]:
# 4) Schema context helpers

def _safe_import_schema_reader():
    # add project root to path for relative imports while running from docs/
    repo_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == 'docs' else os.getcwd()
    if repo_root not in sys.path:
        sys.path.insert(0, repo_root)
    try:
        from schema_reader import get_sql_database_schema_context, refresh_schema_cache  # type: ignore
        return get_sql_database_schema_context, refresh_schema_cache
    except Exception as e:
        raise ImportError('Unable to import schema helpers from schema_reader.py') from e

get_schema_ctx, refresh_cache = _safe_import_schema_reader()
if REFRESH_SCHEMA:
    try:
        path = refresh_cache()
        print('[INFO] Schema cache refreshed at', path)
    except Exception as e:
        print('[WARN] Schema refresh failed:', e)

schema_preview = get_schema_ctx()
print(schema_preview.split('\n')[0])  # first line only

DATABASE: CONTOSO-FI (Azure SQL)


## Optional reasoning step
Ask the model to outline a brief plan before SQL generation. Useful for transparency in demos.

In [7]:
# 5) Reasoning function

def generate_reasoning(intent_entities: str) -> str:
    schema = get_schema_ctx()
    if USING_REASONING:
        prompt_text = REASONING_PROMPT_TEXT.format(schema=schema, intent_entities=intent_entities)
        messages = [{ 'role': 'user', 'content': prompt_text.strip() }]
        return _azure_chat_completions(messages, max_completion_tokens=8192)
    chain = reasoning_prompt | llm
    res = chain.invoke({'schema': schema, 'intent_entities': intent_entities})
    return res.content

## SQL generation
Generate T-SQL for the given intent/entities and schema context.

In [8]:
# 6) SQL generation function

def generate_sql(intent_entities: str) -> str:
    schema = get_schema_ctx()
    if USING_REASONING:
        prompt_text = SQL_PROMPT_TEXT.format(schema=schema, intent_entities=intent_entities)
        messages = [{ 'role': 'user', 'content': prompt_text.strip() }]
        return _azure_chat_completions(messages, max_completion_tokens=8192)
    chain = sql_prompt | llm
    result = chain.invoke({'schema': schema, 'intent_entities': intent_entities})
    return result.content

## SQL sanitization
Extract only the executable SQL portion and normalize quotes for safety.

In [ ]:
# 7) SQL sanitization

def extract_and_sanitize_sql(raw_sql: str) -> str:
    sql_code = raw_sql
    # Prefer fenced code blocks if present (captures full content, e.g., WITH ...)
    m = re.search(r"```sql\s*([\s\S]+?)```", raw_sql, re.IGNORECASE)
    if not m:
        m = re.search(r"```([\s\S]+?)```", raw_sql)
    if m:
        sql_code = m.group(1).strip()
    else:
        # If a CTE exists, start from WITH to capture the entire statement
        with_m = re.search(r"(?is)\bWITH\b\s+[A-Za-z0-9_\[\]]+\s+AS\s*\(", raw_sql)
        if with_m:
            sql_code = raw_sql[with_m.start():].strip()
        else:
            # Fallback to first SELECT
            m2 = re.search(r"(?is)\bSELECT\b[\s\S]+", raw_sql)
            if m2:
                sql_code = m2.group(0).strip()
            else:
                sql_code = raw_sql.strip()
    return (sql_code
            .replace('’', "'")
            .replace('‘', "'")
            .replace('“', '"')
            .replace('”', '"'))

## Execution (optional)
Use `sql_executor.execute_sql_query` to run against Azure SQL. Keep NO_EXEC=True for dry runs during demos.

In [10]:
# 8) SQL execution helpers (definitions only)

def _safe_import_sql_executor():
    repo_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == 'docs' else os.getcwd()
    if repo_root not in sys.path:
        sys.path.insert(0, repo_root)
    try:
        from sql_executor import execute_sql_query  # type: ignore
        return execute_sql_query
    except Exception as e:
        raise ImportError('Unable to import execute_sql_query from sql_executor.py') from e


def _format_table(rows):
    if not rows:
        return 'No results returned.\n', []
    columns = list(rows[0].keys())
    col_widths = {c: max(len(c), max(len(str(r[c])) for r in rows)) for c in columns}
    header = ' | '.join(c.ljust(col_widths[c]) for c in columns)
    sep = '-+-'.join('-' * col_widths[c] for c in columns)
    lines = [header, sep]
    for r in rows:
        lines.append(' | '.join(str(r[c]).ljust(col_widths[c]) for c in columns))
    return ('\n'.join(lines) + '\n', [header, sep] + [' | '.join(str(r[c]).ljust(col_widths[c]) for c in columns) for r in rows])

# Note: Execution is controlled by the global NO_EXEC flag in the Demo settings cell.
# This cell only defines helpers; the end-to-end runner performs the actual execution.

## End-to-end demo cell
Update toggles (`REFRESH_SCHEMA`, `SHOW_REASONING`, `NO_EXEC`) and `test_query` above as needed.
Run this cell to repeat the flow with a different question.

In [ ]:
# 9) Rerun flow with another question (single, hardened output)

# Simple de-duplication guard so each section prints at most once per run
_printed_labels = set()

def print_once(label: str, content: str, color: str):
    if label in _printed_labels:
        return
    _printed_labels.add(label)
    print_colored_block(label, content, color)

# Pick your test query (uses global default unless you set a new one here)
user_query = default_test_query

# 1) Intent/entities
intent_entities = parse_nl_query(user_query)
if SHOW_INTENT:
    print_once('INTENT ENTITIES:', intent_entities[:200] + ('...' if len(intent_entities)>200 else ''), YELLOW)

# 2) Optional reasoning
if SHOW_REASONING:
    try:
        r = generate_reasoning(intent_entities)
        print_once('REASONING:', (r if (r and r.strip()) else '[no reasoning returned]'), LIGHT_BLUE)
    except Exception as e:
        print('[WARN] Reasoning step failed:', e)

# 3) SQL generation with guard
try:
    raw_sql = generate_sql(intent_entities)
except Exception as e:
    raw_sql = ''
    print('[ERROR] SQL generation failed:', e)

if not raw_sql or not raw_sql.strip():
    print_once('RAW SQL (truncated):', '[empty]', WHITE)
    sql_to_run = ''
    print_once('SANITIZED SQL:', '[empty]', LIGHT_GRAY)
else:
    print_once('RAW SQL (truncated):', (raw_sql[:500] + '...') if len(raw_sql)>500 else raw_sql, WHITE)
    sql_to_run = extract_and_sanitize_sql(raw_sql)
    print_once('SANITIZED SQL:', sql_to_run, LIGHT_GRAY)

# 4) Execution with safety checks
if not NO_EXEC:
    sql_final = sql_to_run if 'sql_to_run' in locals() else ''
    if not sql_final or not sql_final.strip():
        print('[INFO] Skipping execution: SQL is empty.')
    elif not re.search(r'\bselect\b', sql_final, re.IGNORECASE):
        print('[INFO] Skipping execution: No SELECT statement detected.')
    else:
        try:
            execute_sql_query = _safe_import_sql_executor()
            rows = execute_sql_query(sql_final)
            table_text, _ = _format_table(rows)
            print_once('RESULTS:', table_text, GOLD)
        except Exception as e:
            print('[ERROR] Query failed:', e)
else:
    print('[INFO] Execution skipped (NO_EXEC=True)')

INTENT ENTITIES:
{
  "intent": "aggregate_metric",
  "aggregation": "weighted_average",
  "metric": "interest rate",
  "group_by": ["region"],
  "filters": [],
  "time_range": null,
  "weight_field": null
}

REASONING:
- Entities mapping: interest rate → vw_LoanPortfolio.InterestRatePct; region → vw_LoanPortfolio.RegionName; weight → vw_LoanPortfolio.PrincipalAmount.
- Tables/Joins: Use dbo.vw_LoanPortfolio only (no joins needed).
- Aggregations: Compute weighted average interest rate per region as sum(InterestRatePct * PrincipalAmount) / sum(PrincipalAmount). Guard against divide-by-zero and exclude NULL InterestRatePct.
- Filters: None requested; exclude rows with NULL or non-positive PrincipalAmount to ensure valid weighting.
- Order/Limit: Order by RegionName ascending; no limit.
- Assumptions: Include all loans regardless of Status unless directed otherwise (can filter to active/open if needed). InterestRatePct is in percentage points and does not require currency conversion.

RAW